In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models

DATA_DIR = '/kaggle/input/csiro-biomass'
WORKING_DIR = '/kaggle/working'

USE_PRETRAINED_WEIGHTS = True
PRETRAINED_WEIGHTS_DIR = '/kaggle/input/pretrained-weights/pretrained_weights'

torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
print(f"Using pretrained weights: {USE_PRETRAINED_WEIGHTS}")


In [ ]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print("\nFirst few rows of train:")
print(train_df.head())


In [ ]:
target_cols = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
target_names = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']

unique_image_paths = train_df['image_path'].unique()
print(f"Total rows in train.csv: {len(train_df)}")
print(f"Number of unique image paths: {len(unique_image_paths)}")
print(f"Expected: {len(train_df)} / 5 targets = {len(train_df) // 5} unique images")

train_images = []
train_targets = []

for img_path in unique_image_paths:
    targets = train_df[train_df['image_path'] == img_path].set_index('target_name')['target'].to_dict()
    full_img_path = os.path.join(DATA_DIR, img_path)
    train_images.append(full_img_path)
    train_targets.append([targets[col] for col in target_cols])

train_data = pd.DataFrame({
    'image_path': train_images,
    **{col: [t[i] for t in train_targets] for i, col in enumerate(target_cols)}
})

print(f"\nReshaped training data: {train_data.shape}")
print(f"This means we have {train_data.shape[0]} unique images")
print(train_data.head())


In [ ]:
class BiomassDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.target_cols = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
        self.has_targets = all(col in dataframe.columns for col in self.target_cols)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.has_targets:
            targets = torch.FloatTensor([
                self.dataframe.iloc[idx][col] for col in self.target_cols
            ])
        else:
            targets = torch.FloatTensor([0.0] * len(self.target_cols))

        return image, targets


In [ ]:
IMG_SIZE = 256

transform_train = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class AugmentedBiomassDataset(Dataset):
    def __init__(self, dataframe, transform=None, augment=True):
        self.dataframe = dataframe
        self.transform = transform
        self.augment = augment
        self.num_augmentations = 6
        self.target_cols = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
        self.has_targets = all(col in dataframe.columns for col in self.target_cols)

    def __len__(self):
        if self.augment:
            return len(self.dataframe) * self.num_augmentations
        return len(self.dataframe)

    def __getitem__(self, idx):
        if self.augment:
            actual_idx = idx // self.num_augmentations
            aug_type = idx % self.num_augmentations
        else:
            actual_idx = idx
            aug_type = 0

        img_path = self.dataframe.iloc[actual_idx]['image_path']
        image = Image.open(img_path).convert('RGB')

        if self.augment and aug_type > 0:
            if aug_type == 1:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
            elif aug_type == 2:
                image = image.transpose(Image.FLIP_TOP_BOTTOM)
            elif aug_type == 3:
                image = image.rotate(90, expand=True)
            elif aug_type == 4:
                image = image.rotate(180, expand=True)
            elif aug_type == 5:
                image = image.rotate(270, expand=True)

        if self.transform:
            image = self.transform(image)

        if self.has_targets:
            targets = torch.FloatTensor([
                self.dataframe.iloc[actual_idx][col] for col in self.target_cols
            ])
        else:
            targets = torch.FloatTensor([0.0] * len(self.target_cols))

        return image, targets

print(f"Full training data: {len(train_data)} images")


In [ ]:
batch_size = 16
num_workers = 2 if torch.cuda.is_available() else 0
print(f"Batch size: {batch_size}")


In [ ]:
def load_pretrained_weights(model, weights_path):
    if os.path.exists(weights_path):
        try:
            state_dict = torch.load(weights_path, map_location='cpu')
            if 'state_dict' in state_dict:
                state_dict = state_dict['state_dict']
            if any(k.startswith('module.') for k in state_dict.keys()):
                state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}

            filtered_dict = {}
            for k, v in state_dict.items():
                if not (k.startswith('fc.') or k.startswith('classifier.')):
                    filtered_dict[k] = v

            model.load_state_dict(filtered_dict, strict=False)
            print(f"Loaded pretrained weights from {os.path.basename(weights_path)}")
            return True
        except Exception as e:
            print(f"Error loading weights from {weights_path}: {e}")
            return False
    return False

class BiomassPredictorResNet18(nn.Module):
    def __init__(self, num_targets=5, pretrained_weights_path=None):
        super(BiomassPredictorResNet18, self).__init__()
        self.backbone = models.resnet18(weights=None)
        num_features = self.backbone.fc.in_features
        if pretrained_weights_path:
            load_pretrained_weights(self.backbone, pretrained_weights_path)
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_targets)
        )
    def forward(self, x):
        return self.backbone(x)

class BiomassPredictorResNet34(nn.Module):
    def __init__(self, num_targets=5, pretrained_weights_path=None):
        super(BiomassPredictorResNet34, self).__init__()
        self.backbone = models.resnet34(weights=None)
        num_features = self.backbone.fc.in_features
        if pretrained_weights_path:
            load_pretrained_weights(self.backbone, pretrained_weights_path)
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_targets)
        )
    def forward(self, x):
        return self.backbone(x)

class BiomassPredictorResNet50(nn.Module):
    def __init__(self, num_targets=5, pretrained_weights_path=None):
        super(BiomassPredictorResNet50, self).__init__()
        self.backbone = models.resnet50(weights=None)
        num_features = self.backbone.fc.in_features
        if pretrained_weights_path:
            load_pretrained_weights(self.backbone, pretrained_weights_path)
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_targets)
        )
    def forward(self, x):
        return self.backbone(x)

class BiomassPredictorEfficientNet(nn.Module):
    def __init__(self, num_targets=5, pretrained_weights_path=None):
        super(BiomassPredictorEfficientNet, self).__init__()
        self.backbone = models.efficientnet_b0(weights=None)
        num_features = self.backbone.classifier[1].in_features
        if pretrained_weights_path:
            load_pretrained_weights(self.backbone, pretrained_weights_path)
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_targets)
        )
    def forward(self, x):
        return self.backbone(x)

print("Model architectures defined!")


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

pretrained_paths = {}
if USE_PRETRAINED_WEIGHTS and os.path.exists(PRETRAINED_WEIGHTS_DIR):
    print(f"Found pretrained weights directory: {PRETRAINED_WEIGHTS_DIR}")
    weight_files = {
        'ResNet18': 'resnet18-f37072fd.pth',
        'ResNet34': 'resnet34-b627a593.pth',
        'ResNet50': 'resnet50-0676ba61.pth',
        'EfficientNet_B0': 'efficientnet_b0_rwightman-3dd342df.pth',
    }
    for model_name, filename in weight_files.items():
        weight_path = os.path.join(PRETRAINED_WEIGHTS_DIR, filename)
        if os.path.exists(weight_path):
            pretrained_paths[model_name] = weight_path
            print(f"  Found {model_name} weights: {filename}")
        else:
            print(f"  Warning: {model_name} weights not found")
else:
    print(f"Pretrained weights directory not found: {PRETRAINED_WEIGHTS_DIR}")
    USE_PRETRAINED_WEIGHTS = False

models_to_train = {}
for model_name in ['ResNet18', 'ResNet34', 'ResNet50', 'EfficientNet_B0']:
    weight_path = pretrained_paths.get(model_name) if USE_PRETRAINED_WEIGHTS else None

    if model_name == 'ResNet18':
        models_to_train[model_name] = BiomassPredictorResNet18(num_targets=5, pretrained_weights_path=weight_path)
    elif model_name == 'ResNet34':
        models_to_train[model_name] = BiomassPredictorResNet34(num_targets=5, pretrained_weights_path=weight_path)
    elif model_name == 'ResNet50':
        models_to_train[model_name] = BiomassPredictorResNet50(num_targets=5, pretrained_weights_path=weight_path)
    elif model_name == 'EfficientNet_B0':
        models_to_train[model_name] = BiomassPredictorEfficientNet(num_targets=5, pretrained_weights_path=weight_path)

print(f"Initialized {len(models_to_train)} models: {list(models_to_train.keys())}")


In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, targets in train_loader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)


def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, targets in val_loader:
            images = images.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            outputs = model(images)
            loss = criterion(outputs, targets)
            running_loss += loss.item()
    return running_loss / len(val_loader)


def train_model(model, model_name, train_loader, val_loader, num_epochs=30, lr=0.001, device='cuda'):
    model = model.to(device)
    criterion = nn.HuberLoss(delta=1.0)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)

    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss = validate(model, val_loader, criterion, device)
        scheduler.step()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()

        print(f"{model_name} Epoch {epoch+1}/{num_epochs} - Train {train_loss:.4f} Val {val_loss:.4f} LR {optimizer.param_groups[0]['lr']:.6f}")

    return best_model_state, best_val_loss

print('Training functions defined!')


In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
model_states = {}
val_losses = {}
num_epochs = 30
initial_lr = 0.001

full_train_data = train_data.copy()

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(full_train_data)):
    fold_train_data = full_train_data.iloc[train_idx].reset_index(drop=True)
    fold_val_data = full_train_data.iloc[val_idx].reset_index(drop=True)

    fold_train_dataset = AugmentedBiomassDataset(fold_train_data, transform=transform_train, augment=True)
    fold_val_dataset = BiomassDataset(fold_val_data, transform=transform_val)

    fold_train_loader = DataLoader(
        fold_train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    fold_val_loader = DataLoader(
        fold_val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )

    print(f"Fold {fold_idx + 1} - Train: {len(fold_train_dataset)}, Val: {len(fold_val_dataset)}")

    for model_name in models_to_train.keys():
        weight_path = pretrained_paths.get(model_name) if USE_PRETRAINED_WEIGHTS else None

        if model_name == 'ResNet18':
            model = BiomassPredictorResNet18(num_targets=5, pretrained_weights_path=weight_path)
        elif model_name == 'ResNet34':
            model = BiomassPredictorResNet34(num_targets=5, pretrained_weights_path=weight_path)
        elif model_name == 'ResNet50':
            model = BiomassPredictorResNet50(num_targets=5, pretrained_weights_path=weight_path)
        elif model_name == 'EfficientNet_B0':
            model = BiomassPredictorEfficientNet(num_targets=5, pretrained_weights_path=weight_path)

        full_model_name = f"{model_name}_fold{fold_idx + 1}"
        model_state, val_loss = train_model(
            model,
            full_model_name,
            fold_train_loader,
            fold_val_loader,
            num_epochs=num_epochs,
            lr=initial_lr,
            device=device,
        )

        model_states[full_model_name] = model_state
        val_losses[full_model_name] = val_loss
        torch.save(model_state, f'{WORKING_DIR}/best_model_{full_model_name}.pth')

print(f"All K-Fold models trained! Total models: {len(model_states)}")


In [ ]:
class EnsembleModel:
    def __init__(self, models_dict, model_states, val_losses, device):
        self.models = {}
        self.device = device
        if val_losses:
            inv_losses = {name: 1.0 / (loss + 1e-8) for name, loss in val_losses.items()}
            total_inv = sum(inv_losses.values())
            self.weights = {name: inv_loss / total_inv for name, inv_loss in inv_losses.items()}
        else:
            num_models = len(model_states)
            self.weights = {name: 1.0 / num_models for name in model_states.keys()}

        for name, model_state in model_states.items():
            base_name = name.split('_fold')[0]
            if base_name in models_dict:
                weight_path = pretrained_paths.get(base_name) if USE_PRETRAINED_WEIGHTS else None
                if base_name == 'ResNet18':
                    model = BiomassPredictorResNet18(num_targets=5, pretrained_weights_path=weight_path).to(device)
                elif base_name == 'ResNet34':
                    model = BiomassPredictorResNet34(num_targets=5, pretrained_weights_path=weight_path).to(device)
                elif base_name == 'ResNet50':
                    model = BiomassPredictorResNet50(num_targets=5, pretrained_weights_path=weight_path).to(device)
                elif base_name == 'EfficientNet_B0':
                    model = BiomassPredictorEfficientNet(num_targets=5, pretrained_weights_path=weight_path).to(device)
                else:
                    continue

                model.load_state_dict(model_state)
                model.eval()
                self.models[name] = model

    def predict(self, dataloader, use_tta=True):
        all_preds = []
        with torch.no_grad():
            for images, _ in dataloader:
                images = images.to(self.device, non_blocking=True)

                if use_tta:
                    tta_preds = []

                    model_preds_weighted = []
                    for name, model in self.models.items():
                        pred = model(images).cpu().numpy()
                        weight = self.weights.get(name, 1.0 / len(self.models))
                        model_preds_weighted.append(pred * weight)
                    tta_preds.append(np.sum(model_preds_weighted, axis=0))

                    images_hflip = torch.flip(images, [3])
                    model_preds_weighted = []
                    for name, model in self.models.items():
                        pred = model(images_hflip).cpu().numpy()
                        weight = self.weights.get(name, 1.0 / len(self.models))
                        model_preds_weighted.append(pred * weight)
                    tta_preds.append(np.sum(model_preds_weighted, axis=0))

                    images_vflip = torch.flip(images, [2])
                    model_preds_weighted = []
                    for name, model in self.models.items():
                        pred = model(images_vflip).cpu().numpy()
                        weight = self.weights.get(name, 1.0 / len(self.models))
                        model_preds_weighted.append(pred * weight)
                    tta_preds.append(np.sum(model_preds_weighted, axis=0))

                    images_90 = torch.rot90(images, k=1, dims=[2, 3])
                    model_preds_weighted = []
                    for name, model in self.models.items():
                        pred = model(images_90).cpu().numpy()
                        weight = self.weights.get(name, 1.0 / len(self.models))
                        model_preds_weighted.append(pred * weight)
                    tta_preds.append(np.sum(model_preds_weighted, axis=0))

                    images_180 = torch.rot90(images, k=2, dims=[2, 3])
                    model_preds_weighted = []
                    for name, model in self.models.items():
                        pred = model(images_180).cpu().numpy()
                        weight = self.weights.get(name, 1.0 / len(self.models))
                        model_preds_weighted.append(pred * weight)
                    tta_preds.append(np.sum(model_preds_weighted, axis=0))

                    images_270 = torch.rot90(images, k=3, dims=[2, 3])
                    model_preds_weighted = []
                    for name, model in self.models.items():
                        pred = model(images_270).cpu().numpy()
                        weight = self.weights.get(name, 1.0 / len(self.models))
                        model_preds_weighted.append(pred * weight)
                    tta_preds.append(np.sum(model_preds_weighted, axis=0))

                    images_hflip_90 = torch.rot90(images_hflip, k=1, dims=[2, 3])
                    model_preds_weighted = []
                    for name, model in self.models.items():
                        pred = model(images_hflip_90).cpu().numpy()
                        weight = self.weights.get(name, 1.0 / len(self.models))
                        model_preds_weighted.append(pred * weight)
                    tta_preds.append(np.sum(model_preds_weighted, axis=0))

                    images_vflip_90 = torch.rot90(images_vflip, k=1, dims=[2, 3])
                    model_preds_weighted = []
                    for name, model in self.models.items():
                        pred = model(images_vflip_90).cpu().numpy()
                        weight = self.weights.get(name, 1.0 / len(self.models))
                        model_preds_weighted.append(pred * weight)
                    tta_preds.append(np.sum(model_preds_weighted, axis=0))

                    ensemble_pred = np.mean(tta_preds, axis=0)
                else:
                    model_preds_weighted = []
                    for name, model in self.models.items():
                        pred = model(images).cpu().numpy()
                        weight = self.weights.get(name, 1.0 / len(self.models))
                        model_preds_weighted.append(pred * weight)
                    ensemble_pred = np.sum(model_preds_weighted, axis=0)

                all_preds.append(ensemble_pred)
        return np.concatenate(all_preds)

ensemble = EnsembleModel(models_to_train, model_states, val_losses, device)
print(f"Ensemble created with {len(ensemble.models)} models")


In [ ]:
test_images = test_df.groupby('image_path').first().reset_index()
test_images['image_path'] = test_images['image_path'].apply(lambda x: os.path.join(DATA_DIR, x))

test_dataset = BiomassDataset(test_images, transform=transform_val)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

all_test_preds = ensemble.predict(test_loader, use_tta=True)

submission = []
for _, row in test_df.iterrows():
    image_path = row['image_path']
    target_name = row['target_name']

    full_img_path = os.path.join(DATA_DIR, image_path)
    img_idx = test_images[test_images['image_path'] == full_img_path].index[0]
    target_idx = target_names.index(target_name)
    prediction = all_test_preds[img_idx, target_idx]

    submission.append({'sample_id': row['sample_id'], 'target': max(0, float(prediction))})

submission_df = pd.DataFrame(submission)
submission_df.to_csv(f'{WORKING_DIR}/submission.csv', index=False)
print('Submission saved to:', f'{WORKING_DIR}/submission.csv')
print(submission_df.head())
